<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=e063dc8bef6c28f58ec672053bbf45b946eb50bc8d8202ddd8392bbf5cc8ffac
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-29 11:47:06
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.36 C
-------------------
Today PnL: -16.32 K (-0.12%)
Current PnL: -23.34 L (-15.7%)
CY Booked + Current PnL: -11.90 L (-8.0%)
-------------------
Total profit:  89.56 K
Total loss:  -24.23 L
-------------------
Total Booked + Current PnL: 15.08 L (12.39%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.4%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 89.21 L (65.47%)
Deployed:  1.22 C
Current:  1.36 C
CAGR/XIRR %: 6.81%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,93.17,35.0,M-SC,2.44,83185.0,-17592.0,17685.0,0.01,-17.46,21.26,0.09,245.0,-0.99,0.61,9.29,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.78,48.0,H-MC,3.63,189510.0,24840.0,18496.0,0.26,15.08,9.76,26.31,89.0,1.34,1.39,24.27,X40N,NTT,AC
50,NATIONALUM,244.55,-46.15,51.0,H-MC,11.85,107866.0,7362.0,21746.0,1.47,7.33,20.16,28.96,79.0,0.34,0.79,43.78,MH,ATH,METALS
42,ITC,452.00,-38.77,45.0,H-LC,1.92,198094.0,-2044.0,22939.0,0.00,-1.02,11.58,10.44,4.0,-0.09,1.45,4.49,X40,NTT,FMCG
57,RELIANCE,1533.00,-14.04,44.0,H-LC,4.07,214906.0,4540.0,24241.0,0.38,2.16,11.28,13.68,37.0,0.19,1.58,19.04,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-178.41,71.0,M-SC,6.64,90564.0,-11646.0,105036.0,9.52,-11.39,115.98,91.37,208.0,-0.11,0.66,47.01,XY25,NTT,FINANCE
78,UJJIVANSFB,60.00,94.87,53.0,H-SC,10.37,122850.0,-19629.0,39152.0,2.99,-13.78,31.87,13.70,163.0,-0.50,0.90,45.00,OX40N,NTT,BANKS
32,HINDZINC,730.22,23.40,57.0,M-LC,2.93,200687.0,-4389.0,116960.0,2.73,-2.14,58.28,54.89,52.0,-0.04,1.47,21.44,X5K,ATH,METALS
10,BANDHANBNK,400.00,-14.68,42.0,H-SC,4.13,213033.0,-65526.0,322958.0,2.55,-23.52,151.60,92.42,151.0,-0.20,1.56,23.12,XY24,NTT,BANKS
21,COFFEEDAY,80.00,-45.67,44.0,L-SC,40.33,79430.0,-34119.0,74172.0,2.32,-30.05,93.38,35.27,268.0,-0.46,0.58,94.41,XR,NTT,HOTELS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,TANLA,1963.11,-20.74,52.0,H-SC,16.90,212150.0,-58611.0,388553.0,-3.80,-21.65,183.15,121.86,133.0,-0.15,1.56,65.96,AR,ATH,IT
26,GREENPANEL,537.00,193.28,39.0,M-SC,0.99,131004.0,-52074.0,131056.0,-2.74,-28.44,100.04,43.14,230.0,-0.40,0.96,23.23,XY24,NTT,MISC
4,ANGELONE,3033.00,-1.02,29.0,H-SC,11.13,175185.0,-15821.0,73525.0,-2.67,-8.28,41.97,30.21,157.0,-0.22,1.29,10.87,X40N,NTT,FINANCE
55,RAJOOENG,143.10,-44.89,21.0,H-SC,14.22,83410.0,-19090.0,59688.0,-2.47,-18.62,71.56,39.61,114.0,-0.32,0.61,0.00,AR,ATH,MISC
38,INDIGOPNTS,1408.00,101.45,29.0,M-SC,1.53,132072.0,-42487.0,42514.0,-2.34,-24.34,32.19,0.02,221.0,-1.00,0.97,14.88,OX40N,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.0,-46.35,27.0,M-SC,1.33,90645.0,-854.0,79604.0,-1.95,-0.93,87.82,86.07,223.0,-0.01,0.67,31.63,XR,NTT,DURABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,94.87,53.0,H-SC,10.37,122850.0,-19629.0,39152.0,2.99,-13.78,31.87,13.70,163.0,-0.50,0.90,45.00,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.17,35.0,M-SC,2.44,83185.0,-17592.0,17685.0,0.01,-17.46,21.26,0.09,245.0,-0.99,0.61,9.29,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,50.0,H-SC,1.53,220725.0,-48942.0,85266.0,-0.54,-18.15,38.63,13.47,138.0,-0.57,1.62,11.64,XY24,NTT,PAINTS
18,CERA,9475.0,-23.15,34.0,H-SC,1.72,140116.0,-35787.0,77806.0,-1.42,-20.34,55.53,23.89,149.0,-0.46,1.03,21.01,OX40N,NTT,CERAMICS
66,SIS,528.0,2079.79,40.0,H-SC,4.02,87975.0,-23057.0,46662.0,-0.06,-20.77,53.04,21.26,156.0,-0.49,0.65,18.86,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,44.35,55.0,M-MC,10.71,228010.0,3048.0,164600.0,1.06,1.35,72.19,74.53,192.0,0.02,1.67,34.05,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,9.78,48.0,H-MC,3.63,189510.0,24840.0,18496.0,0.26,15.08,9.76,26.31,89.0,1.34,1.39,24.27,X40N,NTT,AC
84,VOLTAS,1530.00,-3.04,43.0,H-MC,2.85,203400.0,11658.0,26096.0,1.18,6.08,12.83,19.69,99.0,0.45,1.49,13.29,XY25,NTT,AC
17,CAMS,5211.76,-6.71,42.0,H-SC,1.81,106159.0,4155.0,39767.0,-0.23,4.07,37.46,43.06,122.0,0.10,0.78,21.89,X40N,ATH,MISC
79,UNIONBANK,163.00,-14.73,51.0,M-LC,6.99,153620.0,12780.0,31707.0,0.77,9.07,20.64,31.59,66.0,0.40,1.13,37.97,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-46.15,51.0,H-MC,11.85,107866.0,7362.0,21746.0,1.47,7.33,20.16,28.96,79.0,0.34,0.79,43.78,MH,ATH,METALS
11,BANKINDIA,190.00,-29.19,54.0,H-MC,8.43,181868.0,2060.0,110358.0,1.65,1.15,60.68,62.52,88.0,0.02,1.34,35.62,XR,NTT,BANKS
37,INDIAMART,4850.92,-55.20,25.0,H-SC,9.99,125080.0,1744.0,132022.0,-0.29,1.41,105.55,108.45,119.0,0.01,0.92,24.08,AR,ATH,MISC
85,WHIRLPOOL,2270.00,-46.35,27.0,M-SC,1.33,90645.0,-854.0,79604.0,-1.95,-0.93,87.82,86.07,223.0,-0.01,0.67,31.63,XR,NTT,DURABLES
86,WIPRO,420.00,-16.06,36.0,M-LC,5.47,147659.0,-3286.0,112738.0,1.02,-2.18,76.35,72.51,53.0,-0.03,1.08,3.71,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,29.0,H-LC,12.08,272515.0,-73441.0,148303.0,-1.97,-21.23,54.42,21.64,1.0,-0.50,2.00,0.00,X40,ATH,IT
5,ASIANPAINT,3465.66,-17.90,30.0,H-LC,6.54,207231.0,-44537.0,97751.0,0.52,-17.69,47.17,21.13,27.0,-0.46,1.52,11.47,X40,ATH,PAINTS
44,JIOFIN,387.00,-5.85,31.0,H-LC,12.80,255936.0,-11429.0,76883.0,0.59,-4.27,30.04,24.48,48.0,-0.15,1.88,48.35,XY24,BTT,FINANCE
30,HCLTECH,1943.91,-4.95,33.0,H-LC,10.04,217429.0,-24495.0,87754.0,-0.75,-10.13,40.36,26.15,8.0,-0.28,1.60,2.75,X40,ATH,IT
83,VBL,671.64,-20.64,33.0,H-LC,8.50,286015.0,-29827.0,142493.0,0.80,-9.44,49.82,35.67,5.0,-0.21,2.10,3.10,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-7.51,41.0,H-LC,1.58,229612.0,-16539.0,95748.0,0.54,-6.72,41.70,32.18,16.0,-0.17,1.69,26.90,X200,ATH,IT
42,ITC,452.0,-38.77,45.0,H-LC,1.92,198094.0,-2044.0,22939.0,0.00,-1.02,11.58,10.44,4.0,-0.09,1.45,4.49,X40,NTT,FMCG
65,SIEMENS,4671.5,-3.04,43.0,H-LC,2.40,155775.0,-30320.0,77794.0,0.62,-16.29,49.94,25.51,15.0,-0.39,1.14,14.79,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-14.04,44.0,H-LC,4.07,214906.0,4540.0,24241.0,0.38,2.16,11.28,13.68,37.0,0.19,1.58,19.04,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-22.52,40.0,H-LC,4.29,206692.0,2192.0,39230.0,0.62,1.07,18.98,20.25,10.0,0.06,1.52,9.90,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,29.0,H-LC,12.08,272515.0,-73441.0,148303.0,-1.97,-21.23,54.42,21.64,1.0,-0.50,2.00,0.00,X40,ATH,IT
80,UNITDSPR,1644.00,-14.66,43.0,H-LC,7.21,223858.0,-10286.0,58919.0,0.32,-4.39,26.32,20.77,86.0,-0.17,1.64,2.22,X40N,NTT,BREWERIES
30,HCLTECH,1943.91,-4.95,33.0,H-LC,10.04,217429.0,-24495.0,87754.0,-0.75,-10.13,40.36,26.15,8.0,-0.28,1.60,2.75,X40,ATH,IT
83,VBL,671.64,-20.64,33.0,H-LC,8.50,286015.0,-29827.0,142493.0,0.80,-9.44,49.82,35.67,5.0,-0.21,2.10,3.10,X40N,ATH,FMCG
42,ITC,452.00,-38.77,45.0,H-LC,1.92,198094.0,-2044.0,22939.0,0.00,-1.02,11.58,10.44,4.0,-0.09,1.45,4.49,X40,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-3.04,43.0,H-LC,2.40,155775.0,-30320.0,77794.0,0.62,-16.29,49.94,25.51,15.0,-0.39,1.14,14.79,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,45.0,H-LC,1.92,198094.0,-2044.0,22939.0,0.00,-1.02,11.58,10.44,4.0,-0.09,1.45,4.49,X40,NTT,FMCG
19,CIPLA,1795.00,-22.52,40.0,H-LC,4.29,206692.0,2192.0,39230.0,0.62,1.07,18.98,20.25,10.0,0.06,1.52,9.90,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-17.90,30.0,H-LC,6.54,207231.0,-44537.0,97751.0,0.52,-17.69,47.17,21.13,27.0,-0.46,1.52,11.47,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.04,44.0,H-LC,4.07,214906.0,4540.0,24241.0,0.38,2.16,11.28,13.68,37.0,0.19,1.58,19.04,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,29.0,H-LC,12.08,272515.0,-73441.0,148303.0,-1.97,-21.23,54.42,21.64,1.0,-0.50,2.00,0.00,X40,ATH,IT
40,INFY,2275.00,-20.67,40.0,H-LC,8.19,309191.0,-3697.0,175373.0,0.19,-1.18,56.72,54.87,3.0,-0.02,2.27,5.39,X40,BTT,IT
42,ITC,452.00,-38.77,45.0,H-LC,1.92,198094.0,-2044.0,22939.0,0.00,-1.02,11.58,10.44,4.0,-0.09,1.45,4.49,X40,NTT,FMCG
83,VBL,671.64,-20.64,33.0,H-LC,8.50,286015.0,-29827.0,142493.0,0.80,-9.44,49.82,35.67,5.0,-0.21,2.10,3.10,X40N,ATH,FMCG
1,ABB,7934.00,-40.45,44.0,H-LC,7.70,248981.0,-12638.0,131860.0,0.26,-4.83,52.96,45.57,7.0,-0.10,1.83,5.94,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,44.0,L-SC,40.33,79430.0,-34119.0,74172.0,2.32,-30.05,93.38,35.27,268.0,-0.46,0.58,94.41,XR,NTT,HOTELS
49,MASFIN,398.61,-20.96,35.0,H-SC,12.28,89880.0,-8100.0,29705.0,-1.01,-8.27,33.05,22.05,152.0,-0.27,0.66,30.65,XR,ATH,FINANCE
61,SAMMAANCAP,326.00,-178.41,71.0,M-SC,6.64,90564.0,-11646.0,105036.0,9.52,-11.39,115.98,91.37,208.0,-0.11,0.66,47.01,XY25,NTT,FINANCE
50,NATIONALUM,244.55,-46.15,51.0,H-MC,11.85,107866.0,7362.0,21746.0,1.47,7.33,20.16,28.96,79.0,0.34,0.79,43.78,MH,ATH,METALS
78,UJJIVANSFB,60.00,94.87,53.0,H-SC,10.37,122850.0,-19629.0,39152.0,2.99,-13.78,31.87,13.70,163.0,-0.50,0.90,45.00,OX40N,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1167.94,63.0,M-SC,13.91,172418.0,-8725.0,54191.0,0.68,-4.82,31.43,25.10,235.0,-0.16,1.27,38.18,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.00,-178.41,71.0,M-SC,6.64,90564.0,-11646.0,105036.0,9.52,-11.39,115.98,91.37,208.0,-0.11,0.66,47.01,XY25,NTT,FINANCE
60,SAIL,228.00,44.35,55.0,M-MC,10.71,228010.0,3048.0,164600.0,1.06,1.35,72.19,74.53,192.0,0.02,1.67,34.05,XY24,BTT,STEEL
81,VAIBHAVGBL,521.00,49.31,45.0,H-SC,5.78,129438.0,-53337.0,164412.0,-1.28,-29.18,127.02,60.77,125.0,-0.32,0.95,15.93,XR,NTT,JEWELLERY
50,NATIONALUM,244.55,-46.15,51.0,H-MC,11.85,107866.0,7362.0,21746.0,1.47,7.33,20.16,28.96,79.0,0.34,0.79,43.78,MH,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.28
1,25,43.75
2,50,75.03


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.72
LC    32.57
MC    23.71
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.27
X40      14.80
X40N     11.44
XR       11.10
XY25     10.24
AR        8.30
OX40N     7.88
X200      1.69
MH        1.60
X5K       1.47
SR        1.21
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.27
H-LC    26.04
H-MC    20.76
M-SC    12.99
M-LC     5.47
M-MC     2.65
L-SC     1.46
L-LC     1.06
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.11,-6.71,41.94
IT,12.82,-20.54,86.26
FINANCE,10.11,-19.22,71.23
MISC,6.98,-21.01,88.29
BANKS,6.21,-17.89,79.90
PAINTS,5.63,-20.42,38.36
ELECTRICAL,5.49,-10.09,48.71
INSURANCE,3.77,-7.80,48.10
AC,3.59,1.16,15.39


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3257919.0,22
XR,1397390.0,14
AR,1201545.0,9
X40,831592.0,10
X40N,648643.0,9
OX40N,574180.0,10
XY25,443404.0,7
SR,257398.0,2
X5K,116960.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3808945.0,29
M-SC,1476207.0,17
H-MC,1278669.0,15
H-LC,1273114.0,15
M-LC,404379.0,4
M-MC,316265.0,2
L-SC,261570.0,3
L-MC,61364.0,1
L-LC,39990.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1257198.0      6
M-SC       XY24       848517.0      7
H-SC       AR         820148.0      5
           XR         812703.0      7
H-MC       XY24       584375.0      4
H-LC       X40        532120.0      5
H-SC       X40N       321271.0      4
M-MC       XY24       316265.0      2
H-SC       OX40N      266249.0      4
           SR         257398.0      2
H-LC       X40N       240642.0      3
H-MC       X40        221313.0      4
H-LC       AR         209654.0      2
H-MC       XY25       180311.0      2
M-LC       XY24       174681.0      2
L-SC       XR         172214.0      2
M-SC       AR         171743.0      2
           OX40N      144739.0      4
           XR         128013.0      2
H-LC       XY25       118067.0      3
M-LC       X5K        116960.0      1
           XR         112738.0      1
H-MC       XR         110358.0      1
M-SC       XY25       105036.0      1
H-LC       X200        95748.0      1
L-SC       OX40N       89356.0      1
H-MC       X40N        86730.0      2
M-SC       X40         78159.0      1
H-LC       XY24        76883.0      1
H-SC       MH          73978.0      1
H-MC       OX40N       73836.0      1
L-MC       XR          61364.0      1
L-LC       XY25        39990.0      1
H-MC       MH          21746.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 54.0 seconds
